for i=1;                    p=n*R1/K    
for i=2,...,(k/R2-1);       p=n/(i(i-1)(i-iR2/K))   
for i=K/R2,....K;           p=n*HW(i)   

这里可以把density看作度分布使用的rom里面存的东西，精度就是位数。

但是度如何选择已经知道了。接下来是如何uniform choice the，里面存的是一个数字。

叫做associator

### LT最重要的就是随机数生成算法。

如果生成矩阵A，在这里只需要关心k。

### 关于esi的产生。
1. 我觉得可以暂时先不考虑，因为无非就是两种，一种是压缩产生的信息矩阵，然后接受端只需要解压缩就可以。
可是如果k很大，就会很奇怪。
2. 另一种做法，就是利用key，作为随机数，然后产生一些随机数的序列信息，然后接受端重现随机数产生，并选择相应的模块。这样的做法是减少传输开销，但是会增加复杂度。
### 关于随机数的产生
没有仔细的研究，基本上都是使用lfsr这个方法，只是修正lfsr支持2××n。

然后关于度选择，之后的associator，利用（nx+y）%prime（k）的方法

说明：
    1. n 就是随机数序列
    2. x随机数range（1，prime（k））
    3. y随机数，range(0,print(k))
    4. primt（k）大于等于k的最大 ，range（k，prime（k）），如果选中就剔除
    5. 相当于一个需要 多个周期才能产生一个

关于度选择 ，简单来说可以分为两种。第一种就是利用比较器加rom，另一种就是纯用rom。
简单倒是很简单。。两个我也都会实现。
### 写verilog。理论上来说这个度选择核associator，是要能够生成python代码，和相应的比较mif文件的

In [1]:
### 构建鲁棒弧度子分布，还有想一下度分布生成。
from math import *
import random
import bisect
import struct
class RSD:
    def __init__(self,k,knpPrecision=20,file_name="A_lt"):
        '''里面使用到的参数，都是根据luby的一个专利文档来de 
        该文档仅仅只有一个参数，剩下的参数，基本上都是来源于经验。
        '''
        self.file_name=file_name
        ##load the parameter 
        

        self.k=k
        ## the precision bit 
        self.knpPrecision=knpPrecision
    def patent_rsd(self,):
        k=self.k
        self.__parameter_init()
        ##R1:start_ripple size 
        self.R1=int(self.kdSFactor*sqrt(self.k))
        ##R2:ripple target size
        self.R2=int(self.knRAdd+self.kdRFactor*k**0.25)
        ### 可能出现不了3中情况，因为R1和R2的选择。
        
        r_kinds=int(self.k/self.R2)-2
        if r_kinds<1:
            r_kinds=1
            raise Exception("not good")
        elif r_kinds>self.k:
            r_kinds=K
            raise Exception("not good")
        
        tail_kinds=1
        tail=self.R2
        while(tail>1):
            tail=tail//2
            tail_kinds+=1

        kinds=tail_kinds+r_kinds
        if kinds>k:
            tail_kinds=k-r_kinds
            kinds=k
        
        ##############
        kind_weight=[0]*kinds
        kind_density=[0]*kinds
        kind_frac=[0]*kinds
        ##weight 1
        kind_weight[0]=1
        kind_frac[0]=self.R1/self.k
        ##
        for i in range(1,r_kinds):
            kind_frac[i]=kind_frac[i-1]+1/(i*(i+1)*(1-(i+1)*self.R2/self.k))
            kind_weight[i]=i+1
        n_power=1
        for i in range(tail_kinds):
            n_power=n_power*2
        print(n_power>self.R2)
        j=2
        for i in range(r_kinds,kinds):
            kind_frac[i]=kind_frac[i-1]+self.fdTFactor*n_power/(j*k)
            kind_weight[i]=int(j*k/n_power)
            j=j*2

        ###精确度问题20位精确度
        modulus=1<<self.knpPrecision
        for i in range(kinds):
            kind_density[i]=int(kind_frac[i]/kind_frac[-1]*modulus)
        ######最大重量
        self.max_weight=max(kind_weight)
        #print(kind_weight)
        #### 计算平均
        sum=kind_weight[0]*kind_density[0]
        for i in range(1,kinds):
            sum=sum+(kind_weight[i]*(kind_density[i]-kind_density[i-1]))
        self.average_weight=sum/modulus
        
        self.kind_density=kind_density
        self.kind_weight=kind_weight
        self.file_name+="_patent"
    def __parameter_init(self):
        '''
        从那个文章里面作者提供的参数,
        '''
        self.knRAdd=2
        self.kdRFactor=2.0

        self.kdSFactor=1.4

        self.fdTFactor=1.6
        
    def lt_rsd(self,c,theta):
        '''
        在lt_isd的基础上叠加一个，修正版本，然后再归一化。
        先进性lt编码。
        @parameter c a constant >0
        @parameter theta fail probility

        '''
        k=self.k
        R=c*log(k/theta)*(k**0.5)
        mid=int(k/R)
        if (mid==1 or mid>k):
            raise Exception("sth is wrong")
        kinds=self.k
        kind_weight=[0]*kinds
        kind_frac=[0]*kinds
        kind_density=[0]*kinds
        modulus=1<<self.knpPrecision
        ### 定义概率
        for i in range(k):
            kind_weight[i]=i+1
            if i==0:
                kind_frac[i]=1/self.k+R/((i+1)*k)
            elif i<=mid-1:
                kind_frac[i]=kind_frac[i-1]+1/(i*(i+1))+R/((i+1)*k)
            elif i==mid:
                kind_frac[i]=kind_frac[i-1]+1/(i*(i+1))+R*log(R/theta)/k
            else:
                kind_frac[i]=kind_frac[i-1]+1/(i*(i+1))+0
            
        modulus=1<<self.knpPrecision
        for i in range(kinds):
            kind_density[i]=int(kind_frac[i]/kind_frac[-1]*modulus)
        ######最大重量
        self.max_weight=max(kind_weight)
        #print(kind_weight)
        #### 计算平均
        sum=kind_weight[0]*kind_density[0]
        for i in range(1,kinds):
            sum=sum+(kind_weight[i]*(kind_density[i]-kind_density[i-1]))
        self.average_weight=sum/modulus
        
        self.kind_density=kind_density
        self.kind_weight=kind_weight
        self.file_name+="_rsd"
        pass
    def lt_isd(self,):
        pass
        k=self.k
        kinds=self.k
        kind_weight=[0]*kinds
        kind_frac=[0]*kinds
        kind_density=[0]*kinds
        modulus=1<<self.knpPrecision
        ### 定义概率
        for i in range(k):
            kind_weight[i]=i+1
            if i==0:
                kind_frac[i]=1/self.k
            else:
                kind_frac[i]=kind_frac[i-1]+1/(i*(i+1))
            
        modulus=1<<self.knpPrecision
        for i in range(kinds):
            kind_density[i]=int(kind_frac[i]/kind_frac[-1]*modulus)
        ######最大重量
        self.max_weight=max(kind_weight)
        #print(kind_weight)
        #### 计算平均
        sum=kind_weight[0]*kind_density[0]
        for i in range(1,kinds):
            sum=sum+(kind_weight[i]*(kind_density[i]-kind_density[i-1]))
        self.average_weight=sum/modulus
        
        self.kind_density=kind_density
        self.kind_weight=kind_weight
        self.file_name+="_isd"

        
    def generate_random_v1(self,seed=10,n=None):
        '''
        generate verilog ,mif,还有编码矩阵A的二进制文件。
        '''                
        #step one generate  A the size of A is n,k
        #理论上来说n可以是任意的数值，为了仿真方便，
        #如果不说默认
        k=self.k
        if not n:
            n=ceil(k+log(k))
        '''
        生成 随机数，然后看落在哪个范围内，使用相应的weight，然后再随机产生一系列的随机数，然后再把相应的
        位置1，把这个最后写进文件里面。 

        '''
        random.seed(seed)
        with open(self.file_name,"wb") as fd:
            ##p 是比k大的一个质数
            p=k
            while (1):
                for i in range (2,int(p**0.5+1)):
                    if p%i==0:
                        p=p+1
                        break
                break
            for _ in range(n):
                print("new_message")
                random_desity=random.randint(0,self.kind_density[-1])
                degree=self.kind_weight[bisect.bisect_left(self.kind_density,random_desity)]
                

                ###随机选择数字
                
                x=random.randint(1,p-1)
                y=random.randint(0,p-1)
                summ=y
                count=0
                new_line=[0]*k
                while (count<degree):

                    if summ>=k:
                        summ=(summ+x)%p
                        continue
                    print(f"summ={summ}")
                    new_line[summ]=1
                    summ=(summ+x)%p
                    count+=1
                print(f"count={count}")
                for i in range(floor(k/8)):
                    ##转换成二进制然后保存再文件再文件中。
                    val=0
                    for j in range(8):
                        
                        now=i*8+j
                        if now>=k or new_line[now]==0:
                            continue
                        else:
                            
                            
                            val=val|(1<<(7-j))
                    print(f"val={val}")
                    fd.write(struct.pack("B",val))
        
    def generate_mif_v1(self,):
        file_name=self.file_name+"v1"+"mif"
        pass
        '''
        方法1需要 较少的ram，因为连续相同的ram并没有存，利用地址映射，也就是一个硬件二分，实现的地址映射。
        需要一个precesion×floor（log2（kinds））的lut也是一个很大的开销。
        需要 floor（log2（kinds））×floor（log2（k））
        方法2 把得到的density作为寻址，也就是需要precesion×floor(log2(k))
        '''
        pass



In [2]:
aa=RSD(32)
aa.patent_rsd()
aa.generate_random_v1(n=100)

True
new_message
summ=30
count=1
val=0
val=0
val=0
val=2
new_message
summ=29
count=1
val=0
val=0
val=0
val=4
new_message
summ=10
summ=28
summ=13
summ=31
summ=16
summ=1
summ=19
summ=4
summ=22
summ=7
summ=25
summ=10
summ=28
summ=13
summ=31
summ=16
summ=1
summ=19
summ=4
summ=22
summ=7
summ=25
summ=10
summ=28
summ=13
summ=31
summ=16
summ=1
summ=19
summ=4
summ=22
summ=7
count=32
val=73
val=36
val=146
val=73
new_message
summ=20
count=1
val=0
val=0
val=8
val=0
new_message
summ=23
summ=6
count=2
val=2
val=0
val=1
val=0
new_message
summ=8
count=1
val=0
val=128
val=0
val=0
new_message
summ=26
summ=18
summ=10
count=3
val=0
val=32
val=32
val=32
new_message
summ=29
summ=13
count=2
val=0
val=4
val=0
val=4
new_message
summ=23
summ=10
count=2
val=0
val=32
val=1
val=0
new_message
summ=15
summ=12
count=2
val=0
val=9
val=0
val=0
new_message
summ=2
summ=27
summ=19
summ=11
summ=3
summ=28
summ=20
summ=12
count=8
val=48
val=24
val=24
val=24
new_message
summ=8
count=1
val=0
val=128
val=0
val=0
new_message
sum

In [35]:
aa.kind_density

[77022,
 452600,
 587423,
 660452,
 708253,
 743307,
 771127,
 794601,
 815466,
 834941,
 854061,
 873978,
 896448,
 925339,
 991065,
 1023928,
 1040360,
 1048576]

In [102]:
bisect.bisect_left(bb,bb[0]+1)

1

In [94]:
help(bisect)



Help on module bisect:

NAME
    bisect - Bisection algorithms.

MODULE REFERENCE
    https://docs.python.org/3.8/library/bisect
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

FUNCTIONS
    bisect = bisect_right(...)
        bisect_right(a, x[, lo[, hi]]) -> index
        
        Return the index where to insert item x in list a, assuming a is sorted.
        
        The return value i is such that all e in a[:i] have e <= x, and all e in
        a[i:] have e > x.  So if x already appears in the list, i points just
        beyond the rightmost x already there
        
        Optional args lo (default 0) and hi (default len(a)) bound the
        slice of a to be searched.
    
    bisect_left(...)
        bise

In [61]:
aa.kind_density

[104,
 524392,
 699155,
 786536,
 838965,
 873918,
 898884,
 917608,
 932172,
 943823,
 953355,
 961299,
 968021,
 973782,
 978775,
 983144,
 986999,
 990426,
 993492,
 996252,
 998748,
 1001018,
 1003090,
 1004990,
 1006737,
 1008351,
 1009844,
 1011231,
 1012523,
 1013728,
 1014855,
 1015912,
 1016905,
 1017840,
 1018721,
 1019553,
 1020340,
 1021086,
 1021794,
 1022466,
 1023105,
 1023714,
 1024295,
 1024849,
 1025379,
 1025885,
 1026370,
 1026835,
 1027281,
 1027709,
 1028120,
 1028515,
 1028896,
 1029262,
 1029615,
 1029956,
 1030284,
 1030601,
 1030908,
 1031204,
 1031491,
 1031768,
 1032036,
 1032296,
 1032548,
 1032793,
 1033030,
 1033260,
 1033484,
 1033701,
 1033912,
 1034117,
 1034316,
 1034510,
 1034699,
 1034883,
 1035062,
 1035237,
 1035407,
 1035573,
 1035735,
 1035893,
 1036047,
 1036197,
 1036344,
 1036488,
 1036628,
 1036765,
 1036899,
 1037030,
 1037158,
 1037283,
 1037405,
 1037525,
 1037643,
 1037758,
 1037870,
 1037981,
 1038089,
 1038195,
 1038298,
 1038400,
 103

In [43]:
summ=0
k=10000
for i in range (1,k):
    summ+=1/i



In [44]:
help(log)

Help on built-in function log in module math:

log(...)
    log(x, [base=math.e])
    Return the logarithm of x to the given base.
    
    If the base not specified, returns the natural logarithm (base e) of x.

